<h1>Identifying Wine Heritage By Chemical Composition</h1>

1. Title of Database: Wine recognition data
        Updated Sept 21, 1998 by C.Blake : Added attribute information

2. Sources:
   (a) Forina, M. et al, PARVUS - An Extendible Package for Data
       Exploration, Classification and Correlation. Institute of Pharmaceutical
       and Food Analysis and Technologies, Via Brigata Salerno,
       16147 Genoa, Italy.

   (b) Stefan Aeberhard, email: stefan@coral.cs.jcu.edu.au
   (c) July 1991


Note that we use global variables throughout this.  A better way to do this for production code would be to make a class, but this is just a small demo.

In [1]:
from numpy import *
from matplotlib.pyplot import *
from scipy.optimize import fmin_bfgs,minimize
import csv
import time as t


# Globals to control parameters
regParam = 0.03
nSPerClass = 40
nS = 3*nSPerClass

class1start = 0
class2start = 59
class3start = 130

data = csv.reader(open('uciWineLocation.csv','r'))
rowInd = 0
inData = []
outData = []
outSData = []
testBin = 3*[0]

# Read the data as strings
# Just continue with global variables - it is a small program
for row in data:
    for ri in range(len(row)):
        row[ri] = float(row[ri])
    inData.append(row[1:])
    for ri in range(1,4):
        testBin[ri-1] = (row[0]==float(ri))
    outData.append(testBin.copy())
    outSData.append(row[0])
    
# Parcel into training data, leaving some out
trainX = array(inData[:nSPerClass]+inData[class2start:class2start+nSPerClass]+inData[class3start:class3start+nSPerClass])
trainY = array(outData[:nSPerClass]+outData[class2start:class2start+nSPerClass]+outData[class3start:class3start+nSPerClass]).astype(float)
trainSY = array(outSData[:nSPerClass]+outSData[class2start:class2start+nSPerClass]+outSData[class3start:class3start+nSPerClass]).astype(float)
nInput = len(trainX[0])
nOutput = 3
nHidden = int(2*nInput/3+nOutput)

print('Number of samples: {}'.format(len(inData)))
print('Number of data vectors: {}'.format(nS))
print('Number of data items: {}'.format(nInput))
print('Number of hidden cells: {}'.format(nHidden))

nW0 = nInput*nHidden
nW1 = nHidden*nOutput
nVar = nW0+nW1+nHidden+nOutput

Number of samples: 178
Number of data vectors: 120
Number of data items: 13
Number of hidden cells: 11


Here is some normalization stuff.

In [2]:
def normalization(col):
    # Normalize a column of input data
    numSamp = float(len(col))
    mu = sum(col)/numSamp
    l2norm = sqrt(sum((col-mu)*(col-mu))/numSamp)
    return mu,l2norm

# Find normalization coefficients for each column
def normalizationConst():
    muVals = []
    normVals = []
    for i in range(nInput):
        mu,nrm = normalization(trainX[:,i])
        muVals.append(mu)
        normVals.append(nrm)
    muVals = array(muVals)
    normVals = array(normVals)
    return muVals,normVals

# Normalize the rows
def normalizeByTrain(inputMatrix):
    muVals, normVals = normalizationConst()
    nSamp = len(inputMatrix[:,0])
    outMatrix = inputMatrix*0.0
    for k in range(nSamp):
        outMatrix[k] = (inputMatrix[k]-muVals)/normVals
    return outMatrix

trainX = normalizeByTrain(trainX)

Initialize weights, feed forward and cost/loss functions.

In [3]:
globalW = random.rand(nVar)

output = eye(nOutput)

def feedForward(x,W):
    W0 = array(W[:nW0]).reshape(nHidden,nInput)
    W1 = array(W[(nW0+nHidden):-nOutput]).reshape(nOutput,nHidden)
    hiddenResult = a1(W0.dot(x)+W[nW0:(nW0+nHidden)])
    z2 = W1.dot(hiddenResult)+W[-nOutput:]
    return a2(z2)

def cost(forward,output,W):
    return 0.5*( dot(forward-output,forward-output)+regParam*0.5*linalg.norm(W)**2 )

Here are the activations and a gradient for a one-hidden-layer network.  Note that we can pick activations by setting the a1,a1p, a2, and a2p variables.

Note also that we just used a sum of costs and grads for our ensemble cost and grad.

In [4]:
def S(X):
    X = X*(abs(X)<10)+10.*(X>=10)-10.*(X<=-10)
    return 1./(1+exp(-X))

def SPrime(X):
    X = X*(abs(X)<10)+10.*(X>=10)-10.*(X<=-10)
    ex = exp(-X)
    return ex/((1.+ex)*(1.+ex))

def relu(x,epsilon=0.0):
    return x*(x>0)+epsilon*x*(x<=0)

def reluPrime(x,epsilon=0.0):
    return (x>0)+epsilon*(x<=0)

def selu(x,lamda=1,alpha=1):
    return lamda*( x*(x>0)+alpha*(exp(x)-1)*(x<=0))

def seluPrime(x,lamda=1,alpha=1):
    return lamda*((x>0)+alpha*exp(x)*(x<=0))

a1 = lambda x: relu(x,epsilon=.1)
a1p = lambda x: reluPrime(x,epsilon=.1)
a2 = S
a2p = SPrime

def gradient(x,out,W):
    W0 = array(W[:nW0]).reshape(nHidden,nInput)
    W1 = array(W[nW0+nHidden:-nOutput]).reshape(nOutput,nHidden)
    grad = W*0.0
    # Get a(z_2)
    z1 = dot(W0,x)+W[nW0:nW0+nHidden]
    z2 = dot(W1,a1(z1))+W[-nOutput:]
    forward = a2(z2)
    err = forward-out
    # gradient for b_1 bias weights
    finalLayerDeriv = a2p(z2)*err
    grad[-nOutput:] = finalLayerDeriv+0.
    # gradient for W_1 weights
    grad[(nW0+nHidden):-nOutput] = outer(finalLayerDeriv,a1(z1)).flatten()
    # gradient for b_0 bias weights
    firstLayerDeriv = finalLayerDeriv.dot(W1)*a1p(z1)
    grad[nW0:(nW0+nHidden)] = firstLayerDeriv+0.
    # gradient for W_0 weights
    grad[:nW0] = outer(firstLayerDeriv,x).flatten()
    return grad+regParam*W

def gradSum(W,X,Y):
    nSamp = len(X[:,0])
    grad = zeros(nVar)
    for i in range(nSamp):
        grad += gradient(X[i,:],Y[i,:],W)
    return grad

def forwardTest(W,x,y):
    return cost(feedForward(x,W),y,W)

def forwardTestAll(W,X,Y):
    tot = 0.
    nSamp = len(X[:,0])
    for i in range(nSamp):
        xSamp = X[i,:]
        ySamp = Y[i,:]
        tot += forwardTest(W,xSamp,ySamp)
    return tot    